In [9]:
from transformers import pipeline
from PIL import Image, ImageChops
import requests
import numpy as np
import scipy
import cv2
import os

IMG_PATH =  os.path.expanduser("~/Desktop/profile.jpg")

In [11]:
# load pipe
pipe = pipeline(task="depth-estimation", model="LiheYoung/depth-anything-large-hf")

# load image
image = Image.open(IMG_PATH)

# inference
depth = pipe(image)["depth"]
depth.save(os.path.expanduser("~/Downloads/depth.jpg"))

In [12]:
def shift_image(img, depth_img, shift_amount=10):
    # Ensure base image has alpha
    img = img.convert("RGBA")
    data = np.array(img)

    # Ensure depth image is grayscale (for single value)
    depth_img = depth_img.convert("L")
    depth_data = np.array(depth_img)
    deltas = np.array((depth_data / 255.0) * float(shift_amount), dtype=int)

    # This creates the transprent resulting image.
    # For now, we're dealing with pixel data.
    shifted_data = np.zeros_like(data)

    width = img.width
    height = img.height

    for y, row in enumerate(deltas):
        width = len(row)
        x = 0
        while x < width:
            dx = row[x]
            if x+dx >= width:
                break
            if x-dx < 0:
                shifted_data[y][x-dx] = [0,0,0,0]
            else:
                shifted_data[y][x-dx] = data[y][x]
            x += 1

    # Convert the pixel data to an image.
    shifted_image = Image.fromarray(shifted_data)

    alphas_image = Image.fromarray(scipy.ndimage.binary_fill_holes(ImageChops.invert(shifted_image.getchannel("A")))).convert("1")
    shifted_image.putalpha(ImageChops.invert(alphas_image))

    return shifted_image

In [13]:
def shift_and_inpaint(path, amount):
    shifted = shift_image(image, depth, shift_amount=amount).save(path)
    org_img = cv2.imread(filename=path)
    print("path")
    print(path)
    damaged_img = cv2.imread(filename=path)
    print(damaged_img.shape)
    
    # get the shape of the image
    height, width = damaged_img.shape[0], damaged_img.shape[1]
     
    # Converting all pixels greater than zero to black while black becomes white
    for i in range(height):
        for j in range(width):
            if damaged_img[i, j].sum() > 0:
                damaged_img[i, j] = 0
            else:
                damaged_img[i, j] = [255, 255, 255]
     
    # saving the mask 
    mask = cv2.cvtColor(damaged_img, cv2.COLOR_BGR2GRAY)
    #cv2.imshow("title", mask)
    #cv2.waitKey(0)

    dst = cv2.inpaint(org_img, mask, 3, cv2.INPAINT_NS)
    # Write the output.
    cv2.imwrite(path, dst)

In [20]:
shift_and_inpaint("/Users/herk/Downloads/cart_left.png", 10)
shift_and_inpaint("/Users/herk/Downloads/cart_right.png", 50)

path
/Users/herk/Downloads/cart_left.png
(446, 669, 3)
path
/Users/herk/Downloads/cart_right.png
(446, 669, 3)
